Source: YellowKing Team

Modified for use in Bengali.AI Speech Recognition competition:

**Files loaded locally and processed**

"We loaded the data from bengaliAI huggingface and made the following adjustments:

1. Normalized using bnunicodenormalizer
2. Tokenized using Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali")
3. Removed special characters
4. Resampled at 16000Hz (required for wav2vec2 training later on)
5. Trimmed silence with the threshhold (amplitude < max_amplitude/30 ). This makes training faster
6. Uploaded to huggingface: https://huggingface.co/Lancelot53"

In [42]:
# %%capture
# !pip install transformers
# !pip install jiwer
# !apt install git-lfs

In [51]:
from huggingface_hub import notebook_login
from datasets import concatenate_datasets, load_dataset
import pandas as pd

In [52]:
train_subset = 'train_subset_3'

train_subset_2 = 'train_subset_4'
# Done 1

In [53]:
df = pd.read_csv(f"train_mp3_subsets/{train_subset}/metadata.csv")
df

,file_name,transcription
0,001f20321efd.mp3,"তৃতীয় শতকে দখলে আসে উত্তর আফ্রিকা, আইবেরীয় উ..."
1,0187b75fede0.mp3,খেজুরের রস আবহাওয়ার উপর নির্ভর করে।
2,01a83c4d1776.mp3,অ্যান্টি-ভেনম দিয়ে চিকিত্সার জন্য তাকে মিশরে ...
3,0407bec501b9.mp3,তিনি হচ্ছেন একজন সাবেক ডাব্লিউডাব্লিউই র নারী ...
4,0450ae05e036.mp3,এটি কেন্ট কাউন্টির সদর দপ্তর ও ডোভার-ডিই মেট্র...
...,...,...
291,fd58f5b52c78.mp3,এ সময়ে তার সন্তান মার্ক বুচার ক্লাবের অধিনায়...
292,fd8b79523e88.mp3,ছবিটি পরিচালনা করেছেন রাকেশ রোশন।
293,fde5d47b9b13.mp3,মুক্তিযুদ্ধ শুরু হলে যুদ্ধে যোগ দেন।
294,fe164605e2a1.mp3,তিনি সেখানে ভর্তি হওয়া সর্বকনিষ্ঠ শিক্ষার্থী।


In [54]:
# All loaded as dataset["train"]
dataset1 = load_dataset("audiofolder", data_dir=f"train_mp3_subsets/{train_subset}/")

Resolving data files: 100%|██████████| 297/297 [00:00<?, ?it/s]


In [55]:
dataset2 = load_dataset("audiofolder", data_dir=f"train_mp3_subsets/{train_subset_2}/")


Resolving data files: 100%|██████████| 297/297 [00:00<?, ?it/s]


In [56]:
dataset1 = dataset1["train"]
dataset2 = dataset2["train"]

assert dataset1.features.type == dataset2.features.type
dataset = concatenate_datasets([dataset1, dataset2])
# dataset["train"]=dataset["train"].filter(lambda x,y: x>y, input_columns=["up_votes","down_votes"])
# dataset["validation"]=dataset["validation"].filter(lambda x,y: x>y, input_columns=["up_votes","down_votes"])


In [30]:
# import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\—\‘\'\‚\“\”\…]'

# def remove_special_characters(batch):
#     batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]) + " "
#     return batch

# dataset["train"]=dataset["train"].map(remove_special_characters)
# dataset["validation"]=dataset["validation"].map(remove_special_characters)



# dataset["train"]=dataset["train"].remove_columns(['up_votes','down_votes','age','gender','accent','locale','segment'])
# dataset["validation"]=dataset["validation"].remove_columns(['up_votes','down_votes','age','gender','accent','locale','segment'])


In [31]:
# !pip install bnunicodenormalizer

In [57]:
from bnunicodenormalizer import Normalizer 
bnorm=Normalizer()

def normalize(batch):
  
    _words = [bnorm(word)['normalized']  for word in batch["transcription"].split()]
    batch["transcription"] =  " ".join([word for word in _words if word is not None])
    return batch

dataset = dataset.map(normalize)

Map:   0%|          | 0/592 [00:00<?, ? examples/s]

Map: 100%|██████████| 592/592 [00:02<00:00, 229.41 examples/s]


In [58]:

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali")

In [59]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 592
})

In [60]:
# dataset["train"][0]["audio"]

KeyError: "Column train not in the dataset. Current columns in the dataset: ['audio', 'transcription']"

In [61]:
import numpy as np

def prepare_dataset(batch):

    batch["audio"]["array"] = np.trim_zeros(batch["audio"]["array"], 'fb')
    audio = batch["audio"]
    

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [62]:
dataset_train = dataset.map(prepare_dataset)
# dataset_train = dataset.map(prepare_dataset, remove_columns=dataset.column_names)
# dataset_validation = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/592 [00:00<?, ? examples/s]

c:\Users\Moses\anaconda3\envs\ml_audio\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|█████████▉| 591/592 [00:24<00:00, 59.46 examples/s]

In [ ]:
def trim_silence(batch):
    arr = batch['input_values']
    
    try:
        _max = max(max(arr), -min(arr))
        old_length = len(arr)
        
        threshold = 30

        for i,e in enumerate(arr):
            if threshold*e>_max:
                break

        for j,e in enumerate(reversed(arr)):
            if threshold*e>_max:
                break

        batch['input_values'] = arr[i:old_length-j]
        batch['input_length'] = old_length -i -j
    except:
        print(batch['input_length'])
    return batch
    



In [ ]:
dataset_train = dataset_train.map(trim_silence)

Map: 100%|██████████| 296/296 [00:11<00:00, 26.07 examples/s]


In [40]:
# dataset_train.push_to_hub("commonVoice_bn_train_preprocessed_trimmed")
# dataset_validation.push_to_hub("commonVoice_bn_validation_preprocessed")

In [ ]:
# Save the dataset
dataset_train = dataset_train.train_test_split(test_size=0.15, seed=4)
dataset_train.save_to_disk(f"{train_subset}_preprocessed_trimmed")

Saving the dataset (1/1 shards): 100%|██████████| 45/45 [00:00<00:00, 1840.95 examples/s]
